In [2]:
!pip install datasets transformers pandasql dask -qqq
# torch.cuda.empty_cache()

In [4]:
model_checkpoint = "t5-small"
model_date = '2021-10-11`'
model_path=f'yazdipour/text-to-sparql-t5-small-2021-10-15_01-00'

In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
prefix = "translate english to sparql: "
  
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")

In [8]:
translator = pipeline(
    "translation_xx_to_yy",
    model=model,
    tokenizer=tokenizer,
    device=0 #0 for cuda, -1 for cpu
)

In [9]:
def replace_all(text, dict):
    for i, j in dict.items():
        text = text.replace(i, j)
    return text

In [10]:
def decode_props(qry):
    qry = qry.replace('[','{').replace(']','}')
    return qry

In [11]:
translate= lambda q: decode_props(translator(prefix+q, max_length=100)[0]['translation_text'])

In [12]:
!pip install beautifultable -qqq

In [13]:
from beautifultable import BeautifulTable
def pretty_translate(t,q): 
    ans = translate(q)
    # print('QUESTION:\n' + q + '\nTARGET:\n'+ t +'\nRESULT:\n' + ans[0]+ '\nRESULT-DECODED:\n' + ans[1])
    table = BeautifulTable(maxwidth=140)
    # table.column_headers = ["", "Man Utd","Man City","T Hotspur"]
    table.rows.append(['QUESTION', q])
    table.rows.append(['Target', t])
    table.rows.append(['RESULT-Raw', ans])
    print(table)

# TESTING

In [14]:
prefix

'translate en to sparql: '

In [15]:
translator(prefix+'Who is Marlin Manson?', max_length=100)

[{'translation_text': 'SELECT ?answer WHERE [ wd:marlin manson> wdt:instance of> ?answer . ?answer wdt:instance of> wd:marlin manson> ]'}]

In [16]:
# <re.Match object; span=(52, 66), match=':marlin_manson'>
# ('select distinct ?sbj where { ?sbj wdt:instance_of wd:marlin_manson . ?sbj wdt:instance_of wd:human }',
#  'select distinct ?sbj where { ?sbj wdt:P31 wd:marlin_manson . ?sbj wdt:P31 wd:Q5 }')
translate('Who is Marlin Manson?')

'SELECT ?answer WHERE { wd:marlin manson> wdt:instance of> ?answer . ?answer wdt:instance of> wd:marlin manson> }'

In [17]:
translate('Who is Bill Gates?')

'SELECT ?answer WHERE { wd:bilbert gates> wdt:instance of> ?answer . ?answer wdt:instance of> wd:bilbert gates>}'

In [18]:
translate('what is?')

'SELECT ?answer WHERE { wd:assailant> wdt:instance of> ?answer . ?answer wdt:instance of> wd:instance of> wd:instance of> }'

In [19]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }','Who is the country for head of state of Mahmoud Abbas?')

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                   Who is the country for head of state of Mahmoud Abbas?                                    |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   |                       select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }                        |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:mahmoud abbas> wdt:country> ?answer . ?answer wdt:head of state> ?answer . ?answer wdt:country> ? |
|            |                                                          answer }                                                           |
+------------

In [20]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }',"What country is Mahmoud Abbas the head of state of?")

+------------+------------------------------------------------------------------------------------------------------+
|  QUESTION  |                         What country is Mahmoud Abbas the head of state of?                          |
+------------+------------------------------------------------------------------------------------------------------+
|   Target   |            select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }            |
+------------+------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:mahmoud abbas> wdt:head of state> ?answer . ?answer wdt:country> ?answer } |
+------------+------------------------------------------------------------------------------------------------------+


In [21]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }','Who is the country for head of state of Mahmoud Abbas?')

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                   Who is the country for head of state of Mahmoud Abbas?                                    |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   |                       select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }                        |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:mahmoud abbas> wdt:country> ?answer . ?answer wdt:head of state> ?answer . ?answer wdt:country> ? |
|            |                                                          answer }                                                           |
+------------

In [22]:
pretty_translate("SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}","Which female actress is the voice over on South Park and is employed as a singer?".lower())

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                      which female actress is the voice over on south park and is employed as a singer?                      |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   |                      SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}                       |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:south park> wdt:voice over> ?answer . ?answer wdt:voice over> ?answer . ?answer wdt:voice over> ? |
|            |                                                          answer }                                                           |
+------------

In [23]:
pretty_translate("SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}","Which female actress on South Park is the voice over and is used as a singer?")

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                        Which female actress on South Park is the voice over and is used as a singer?                        |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   |                      SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}                       |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:south park> wdt:voice over> ?answer . ?answer wdt:voice over> ?answer . ?answer wdt:voice over> ? |
|            |                                                          answer }                                                           |
+------------

In [24]:
# Paul Erdős (Q173746)
pretty_translate("select distinct ?answer where { wd:Q173746 wdt:P3973 ?answer}","Which is the PIM authority ID of Paul Erd?")

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                         Which is the PIM authority ID of Paul Erd?                                          |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   |                                select distinct ?answer where { wd:Q173746 wdt:P3973 ?answer}                                |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:Paul Erd> wdt:authority ID> ?answer . ?answer wdt:authority ID> ?answer . ?answer wdt:authority I |
|            |                                                        D> ?answer }                                                         |
+------------

In [25]:
pretty_translate("SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),'2009')) }", 
            "What was the population of Somalia in 2009-0-0?")

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                       What was the population of Somalia in 2009-0-0?                                       |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   |       SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),'2009')) }        |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:simala> wdt:population> ?answer . ?answer wdt:population> ?answer . ?answer wdt:population> ?answ |
|            |                                                            er }                                                             |
+------------

In [26]:
translate('Humans born in New York City') #random query - answer seems correct

'SELECT ?answer WHERE { wd:new york city> wdt:humans born in> wd:new york city> }'

In [27]:
# From QALD
target = "ASK WHERE { <http://dbpedia.org/resource/Taiko> a <http://dbpedia.org/class/yago/WikicatJapaneseMusicalInstruments> }"
q =  "Are Taiko some kind of Japanese musical instrument?"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                     Are Taiko some kind of Japanese musical instrument?                                     |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   |    ASK WHERE { <http://dbpedia.org/resource/Taiko> a <http://dbpedia.org/class/yago/WikicatJapaneseMusicalInstruments> }    |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:taiko> wdt:musical instrument> ?answer . ?answer wdt:musical instrument> ?answer . ?answer wdt:mu |
|            |                                                 sical instrument> ?answer }                                                 |
+------------

In [28]:
# From QALD
target = "PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT DISTINCT ?uri WHERE { ?uri dct:subject dbc:Assassins_of_Julius_Caesar }"
q =  "Who killed Caesar?"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                                     Who killed Caesar?                                                      |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT DISTINCT ?uri WHERE { ?u |
|            |                                       ri dct:subject dbc:Assassins_of_Julius_Caesar }                                       |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:césar> wdt:mortified> ?answer . ?answer wdt:mortified> ?answer . ?answer wdt:instance of> wd:césa |
|            

In [29]:
# From QALD
q = 'What is the highest mountain in Germany?'
target = "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX onto: <http://dbpedia.org/ontology/>  \nSELECT ?uri WHERE { ?uri rdf:type onto:Mountain ; onto:elevation ?elevation ; onto:locatedInArea <http://dbpedia.org/resource/Germany> } ORDER BY DESC(?elevation) LIMIT 1"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                          What is the highest mountain in Germany?                                           |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX onto: |
|            |                                               <http://dbpedia.org/ontology/>                                                |
|            | SELECT ?uri WHERE { ?uri rdf:type onto:Mountain ; onto:elevation ?elevation ; onto:locatedInArea <http://dbpedia.org/resour |
|            |                                       ce/Germany> } ORDER BY DESC(?elevation) LIMIT 1                                       |
+------------

In [30]:
# From QALD
q = 'Which American presidents were in office during the Vietnam War?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT ?uri WHERE { ?uri dct:subject dbc:Presidents_of_the_United_States . res:Vietnam_War dbo:commander ?uri }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                              Which American presidents were in office during the Vietnam War?                               |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX dct: <http://purl.org/dc/terms |
|            | /> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT ?uri WHERE { ?uri dct:subject dbc:Presidents_of_the_United_St |
|            |                                         ates . res:Vietnam_War dbo:commander ?uri }                                         |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw 

In [31]:
# From QALD
q = 'How many gold medals did Michael Phelps win at the 2008 Olympics?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT Count(?sub) as ?c WHERE { ?sub dbo:goldMedalist dbr:Michael_Phelps . filter (contains (str(?sub), \"2008\") && contains (str(?sub), \"Olympics\")) }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                              How many gold medals did Michael Phelps win at the 2008 Olympics?                              |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/0 |
|            | 2/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT Count(?sub) as ?c WHERE { ?sub dbo:goldMed |
|            |           alist dbr:Michael_Phelps . filter (contains (str(?sub), "2008") && contains (str(?sub), "Olympics")) }            |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw 

In [32]:
# From QALD
q = 'What is the profession of Frank Herbert?'
target = "PREFIX dbpedia2: <http://dbpedia.org/property/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?string WHERE { res:Frank_Herbert dbpedia2:occupation ?string }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                          What is the profession of Frank Herbert?                                           |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | PREFIX dbpedia2: <http://dbpedia.org/property/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?string WHERE {  |
|            |                                       res:Frank_Herbert dbpedia2:occupation ?string }                                       |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:franc hrw> wdt:profession> ?answer . ?answer wdt:profession> ?answer . ?answer wdt:profession> ?a |
|            

In [33]:
# From QALD
q = 'How many seats does the home stadium of FC Porto have?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbp: <http://dbpedia.org/property/> PREFIX dbr: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX db: <http://dbpedia.org/> SELECT ?capacity WHERE { { dbr:FC_Porto dbo:ground ?ground . ?ground dbo:capacity ?capacity } UNION { dbr:FC_Porto dbo:ground ?ground . ?ground dbp:capacity ?capacity } }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                   How many seats does the home stadium of FC Porto have?                                    |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbp: <http://dbpedia.org/property/> PREFIX dbr: <http://dbpedia.org/resou |
|            | rce/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX |
|            |  db: <http://dbpedia.org/> SELECT ?capacity WHERE { { dbr:FC_Porto dbo:ground ?ground . ?ground dbo:capacity ?capacity } UN |
|            |                         ION { dbr:FC_Porto dbo:ground ?ground . ?ground dbp:capacity ?capacity } }                          |
+------------

In [34]:
# From QALD
q = 'Which frequent flyer program has the most airlines?'
target = "SELECT ?uri WHERE { ?airline <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Airline> . ?airline <http://dbpedia.org/property/frequentFlyer> ?uri. } GROUP BY ?uri ORDER BY DESC(COUNT(DISTINCT ?airline)) OFFSET 0 LIMIT 1"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                     Which frequent flyer program has the most airlines?                                     |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | SELECT ?uri WHERE { ?airline <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Airline> . ?air |
|            | line <http://dbpedia.org/property/frequentFlyer> ?uri. } GROUP BY ?uri ORDER BY DESC(COUNT(DISTINCT ?airline)) OFFSET 0 LIM |
|            |                                                            IT 1                                                             |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw 

In [35]:
# From QALD
q = 'Which European countries have a constitutional monarchy?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> PREFIX dbr: <http://dbpedia.org/resource/> SELECT ?uri WHERE { ?uri dct:subject dbc:Countries_in_Europe ; dbo:governmentType dbr:Constitutional_monarchy }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                  Which European countries have a constitutional monarchy?                                   |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource |
|            | /Category:> PREFIX dbr: <http://dbpedia.org/resource/> SELECT ?uri WHERE { ?uri dct:subject dbc:Countries_in_Europe ; dbo:g |
|            |                                         overnmentType dbr:Constitutional_monarchy }                                         |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw 

In [36]:
# From QALD
q = 'Which countries have places with more than two caves?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> SELECT DISTINCT ?uri WHERE { ?cave rdf:type dbo:Cave ; dbo:location ?uri . ?uri rdf:type dbo:Country } GROUP BY ?uri HAVING ( COUNT(?cave) > 2 )"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                    Which countries have places with more than two caves?                                    |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> SELECT DISTINCT ?uri W |
|            | HERE { ?cave rdf:type dbo:Cave ; dbo:location ?uri . ?uri rdf:type dbo:Country } GROUP BY ?uri HAVING ( COUNT(?cave) > 2 )  |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:höhle> wdt:place of birth> ?answer . ?answer wdt:place of birth> ?answer . ?answer wdt:place of b |
|            

In [37]:
# From QALD
q = 'Which airports are located in California, USA?'
target = "SELECT DISTINCT ?uri WHERE { ?uri a <http://dbpedia.org/ontology/Airport> { ?uri <http://dbpedia.org/ontology/location> <http://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/city> <http://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/city> ?city . ?city <http://dbpedia.org/ontology/isPartOf> <http://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/operator> <http://dbpedia.org/resource/California> } }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                       Which airports are located in California, USA?                                        |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | SELECT DISTINCT ?uri WHERE { ?uri a <http://dbpedia.org/ontology/Airport> { ?uri <http://dbpedia.org/ontology/location> <ht |
|            | tp://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/city> <http://dbpedia.org/resource/Califo |
|            | rnia> } UNION { ?uri <http://dbpedia.org/ontology/city> ?city . ?city <http://dbpedia.org/ontology/isPartOf> <http://dbpedi |
|            | a.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/operator> <http://dbpedia.org/resource/California> } |
|            

In [38]:
# From QALD
q = "What are the nicknames of San Francisco?"
target = "SELECT DISTINCT ?string WHERE { res:San_Francisco foaf:nick ?string }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                          What are the nicknames of San Francisco?                                           |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   |                            SELECT DISTINCT ?string WHERE { res:San_Francisco foaf:nick ?string }                            |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:san francisco> wdt:named after> ?answer . ?answer wdt:named after> ?answer . ?answer wdt:named af |
|            |                                      ter> ?answer . ?answer wdt:named after> ?answer }                                      |
+------------

In [39]:
# From QALD
q = "What is Angela Merkel’s birth name?"
target = "SELECT DISTINCT ?string WHERE { res:Angela_Merkel dbp:birthName ?string }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                             What is Angela Merkel’s birth name?                                             |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   |                          SELECT DISTINCT ?string WHERE { res:Angela_Merkel dbp:birthName ?string }                          |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:anthona faschina> wdt:birth name> ?answer . ?answer wdt:name of birth> ?answer . ?answer wdt:name |
|            |                                                     of birth> ?answer }                                                     |
+------------

In [40]:
# From QALD
q = "Who is the mayor of Berlin?"
target = "SELECT DISTINCT ?uri WHERE { res:Berlin dbp:leader ?uri }"
pretty_translate(target, q)

+------------+------------------------------------------------------------------------------------------------+
|  QUESTION  |                                  Who is the mayor of Berlin?                                   |
+------------+------------------------------------------------------------------------------------------------+
|   Target   |                   SELECT DISTINCT ?uri WHERE { res:Berlin dbp:leader ?uri }                    |
+------------+------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:berlin> wdt:prime of> ?answer . ?answer wdt:instance of> wd:mayor> } |
+------------+------------------------------------------------------------------------------------------------+


In [41]:
# From QALD
q = "Which software has been published by Mean Hamster Software?"
target = "SELECT DISTINCT ?uri WHERE { ?uri rdf:type onto:Software { ?uri prop:publisher \"Mean Hamster Software\"@en } UNION { ?uri onto:publisher res:Mean_Hamster_Software } }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                 Which software has been published by Mean Hamster Software?                                 |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | SELECT DISTINCT ?uri WHERE { ?uri rdf:type onto:Software { ?uri prop:publisher "Mean Hamster Software"@en } UNION { ?uri on |
|            |                                         to:publisher res:Mean_Hamster_Software } }                                          |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:moyen hamster software> wdt:published by> ?answer . ?answer wdt:published by> ?answer . ?answer w |
|            

In [42]:
# From QALD
q = "Which country was Bill Gates born in?"
target = "SELECT DISTINCT ?country WHERE { { dbr:Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:country ?country } UNION { dbr:Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:isPartOf ?place . ?place dbo:country ?country } }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                            Which country was Bill Gates born in?                                            |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | SELECT DISTINCT ?country WHERE { { dbr:Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:country ?country } UNION { d |
|            |        br:Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:isPartOf ?place . ?place dbo:country ?country } }         |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw |                 SELECT ?answer WHERE { wd:bill gates> wdt:country> ?answer . ?answer wdt:country> ?answer }                 |
+------------

In [43]:
# From QALD
q = "How many grand-children did Jacques Cousteau have?"
target = "SELECT COUNT(DISTINCT ?y AS ?y) WHERE { <http://dbpedia.org/resource/Jacques_Cousteau> <http://dbpedia.org/ontology/child> ?x . ?x <http://dbpedia.org/ontology/child> ?y . }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                     How many grand-children did Jacques Cousteau have?                                      |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | SELECT COUNT(DISTINCT ?y AS ?y) WHERE { <http://dbpedia.org/resource/Jacques_Cousteau> <http://dbpedia.org/ontology/child>  |
|            |                                     ?x . ?x <http://dbpedia.org/ontology/child> ?y . }                                      |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:Jacques Cousteau> wdt:grandchildren> ?answer . ?answer wdt:grandchildren> ?answer . ?answer wdt:g |
|            

In [44]:
# From QALD
q = "Give me all professional skateboarders from Sweden."
target = "SELECT DISTINCT ?uri WHERE { ?uri dbo:occupation dbr:Skateboarder { ?uri dbo:birthPlace dbr:Sweden } UNION { ?uri dbo:birthPlace ?place . ?place dbo:country dbr:Sweden } }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                     Give me all professional skateboarders from Sweden.                                     |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | SELECT DISTINCT ?uri WHERE { ?uri dbo:occupation dbr:Skateboarder { ?uri dbo:birthPlace dbr:Sweden } UNION { ?uri dbo:birth |
|            |                                      Place ?place . ?place dbo:country dbr:Sweden } }                                       |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw |                         select distinct ?object where { wd:suède> wdt:professional skateboarder> }                          |
+------------

In [45]:
# From QALD
q = "Which monarchs of the United Kingdom were married to a German?"
target = "SELECT DISTINCT ?uri WHERE { ?uri rdf:type yago:WikicatMonarchsOfTheUnitedKingdom ; dbo:spouse ?spouse . ?spouse dbo:birthPlace res:Germany }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                               Which monarchs of the United Kingdom were married to a German?                                |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | SELECT DISTINCT ?uri WHERE { ?uri rdf:type yago:WikicatMonarchsOfTheUnitedKingdom ; dbo:spouse ?spouse . ?spouse dbo:birthP |
|            |                                                     lace res:Germany }                                                      |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:u.s.> wdt:heirated monarch> ?answer . ?answer wdt:heirated monarch> ?answer . ?answer wdt:instanc |
|            

In [46]:
# From QALD
q = "Give me all Argentine films."
target = "SELECT DISTINCT ?uri WHERE { { ?uri rdf:type yago:ArgentineFilms } UNION { ?uri rdf:type dbo:Film { ?uri dbo:country res:Argentina } UNION { ?uri dbp:country \"Argentina\"@en } } }"
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                                Give me all Argentine films.                                                 |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   | SELECT DISTINCT ?uri WHERE { { ?uri rdf:type yago:ArgentineFilms } UNION { ?uri rdf:type dbo:Film { ?uri dbo:country res:Ar |
|            |                                   gentina } UNION { ?uri dbp:country "Argentina"@en } } }                                   |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT DISTINCT ?subject ?subject_label WHERE { ?subject wdt:instance of> wd:argentina> . ?subject rdfs:label ?subject_labe |
|            

In [47]:
# From QALD
q = "How did Michael Jackson die?"
target = "SELECT DISTINCT ?s WHERE { <http://dbpedia.org/resource/Michael_Jackson> <http://dbpedia.org/property/deathCause> ?s }"
pretty_translate(target, q)

+------------+------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                              How did Michael Jackson die?                                              |
+------------+------------------------------------------------------------------------------------------------------------------------+
|   Target   | SELECT DISTINCT ?s WHERE { <http://dbpedia.org/resource/Michael_Jackson> <http://dbpedia.org/property/deathCause> ?s } |
+------------+------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw |       SELECT ?answer WHERE { wd:fjord jillson> wdt:mortem time> ?answer . ?answer wdt:representation> ?answer }        |
+------------+------------------------------------------------------------------------------------------------------------------------+


In [48]:
# From QALD
q = "Where did Michael Jackson died?"
target = "."
pretty_translate(target, q)

+------------+-----------------------------------------------------------------------------------------------------------------------------+
|  QUESTION  |                                               Where did Michael Jackson died?                                               |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
|   Target   |                                                              .                                                              |
+------------+-----------------------------------------------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:michael jong> wdt:place of death> ?answer . ?answer wdt:place of death> ?answer . ?answer wdt:pla |
|            |                                                   ce of death> ?answer }                                                    |
+------------

In [49]:
# From QALD
q = "Which classes does the Millepede belong to?"
target = "SELECT DISTINCT ?String WHERE { res:Millipede dbp:taxon ?String }"
pretty_translate(target, q)

+------------+----------------------------------------------------------------------------------------+
|  QUESTION  |                      Which classes does the Millepede belong to?                       |
+------------+----------------------------------------------------------------------------------------+
|   Target   |           SELECT DISTINCT ?String WHERE { res:Millipede dbp:taxon ?String }            |
+------------+----------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:millepede> wdt:class> ?answer . ?answer wdt:class> ?answer } |
+------------+----------------------------------------------------------------------------------------+


In [50]:
# From QALD
q = "Which classes does the Millepede belong to?"
target = "SELECT DISTINCT ?String WHERE { res:Millipede dbp:taxon ?String }"
pretty_translate(target, q)

+------------+----------------------------------------------------------------------------------------+
|  QUESTION  |                      Which classes does the Millepede belong to?                       |
+------------+----------------------------------------------------------------------------------------+
|   Target   |           SELECT DISTINCT ?String WHERE { res:Millipede dbp:taxon ?String }            |
+------------+----------------------------------------------------------------------------------------+
| RESULT-Raw | SELECT ?answer WHERE { wd:millepede> wdt:class> ?answer . ?answer wdt:class> ?answer } |
+------------+----------------------------------------------------------------------------------------+
